# Lab 10.1: CNN with MNIST Classifier

**Jonathan Choi 2021**

**[Deep Learning By Torch] End to End study scripts of Deep Learning by implementing code practice with Pytorch.**

If you have an any issue, please PR below.

[[Deep Learning By Torch] - Github @JonyChoi](https://github.com/jonychoi/Deep-Learning-By-Torch)

Here, we are going to build simple CNN model for MNIST Classifying. To more understand about CNN, please refer before script "10.0 - About CNN".

## Imports

In [10]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.nn.init as init
import torchvision.datasets as datasets
import torchvision.transforms as transforms

In [11]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(1)

if device == 'cuda':
    torch.cuda.manual_seed(1)

## Load MNIST Data

In [12]:
mnist_train = datasets.MNIST(root = 'MNIST_data/',
                             download = True,
                             train = True,
                             transform = transforms.ToTensor())
mnist_test = datasets.MNIST(root = 'MNIST_data/',
                            download = True,
                            train = False,
                            transform = transforms.ToTensor())

In [13]:
batch_size = 100
training_epochs = 20
learning_rate = 0.001

In [14]:
data_loader = torch.utils.data.DataLoader(dataset = mnist_train, batch_size = batch_size, shuffle = True, drop_last = True)

## Set Hyperparameters

### Take a Moment!

Let's remind the PyTorch's CNN APIs.

**CONV2D**

> CLASS: ```torch.nn.Conv2d(in_channels, out_channels, kernel_size, stride=1, padding=0, dilation=1, groups=1, bias=True, padding_mode='zeros', device=None, dtype=None)```

Applies a 2D convolution over an input signal composed of several input planes.

In the simplest case, the output value of the layer with input size $(N, C_{in}, H, W)$ and output $(N, C_{out}, H_{out}, W_{out})$  can be precisely described as:

$$\text{out}(N_i, C_{\text{out}_j}) = \text{bias}(C_{\text{out}_j}) + \sum_{k = 0}^{C_{\text{in}} - 1} \text{weight}(C_{\text{out}_j}, k) \star \text{input}(N_i, k)$$


where ⋆ is the valid 2D [cross-correlation](https://en.wikipedia.org/wiki/Cross-correlation) operator, $N$ is a batch size, $C$ denotes a number of channels, $H$ is a height of input planes in pixels, and $W$ is width in pixels.

This module supports [TensorFloat32](https://pytorch.org/docs/stable/notes/cuda.html#tf32-on-ampere).

- ```stride``` controls the stride for the cross-correlation, a single number or a tuple.

- ```padding``` controls the amount of padding applied to the input. It can be either a string {‘valid’, ‘same’} or a tuple of ints giving the amount of implicit padding applied on both sides.

- ```dilation``` controls the spacing between the kernel points; also known as the à trous algorithm. It is harder to describe, but this [link](https://github.com/vdumoulin/conv_arithmetic/blob/master/README.md) has a nice visualization of what dilation does.

- ```groups``` controls the connections between inputs and outputs. in_channels and out_channels must both be divisible by groups. For example,

> - At groups=1, all inputs are convolved to all outputs.

> - At groups=2, the operation becomes equivalent to having two conv layers side by side, each seeing half the input channels and producing half the output channels, and both subsequently concatenated.

> - At groups= in_channels, each input channel is convolved with its own set of filters (of size out_channels / in_channels)

The parameters ```kernel_size```, ```stride```, ```padding```, ```dilation``` can either be:

- a single ```int``` – in which case the same value is used for the height and width dimension

- a ```tuple``` of two ints – in which case, the first int is used for the height dimension, and the second int for the width dimension


**Parameters**

- **in_channels** (int) – Number of channels in the input image

- **out_channels** (int) – Number of channels produced by the convolution

- **kernel_size** (int or tuple) – Size of the convolving kernel

- **stride** (int or tuple, optional) – Stride of the convolution. Default: 1

- **padding** (int, tuple or str, optional) – Padding added to all four sides of the input. Default: 0

- **padding_mode** (string, optional) – 'zeros', 'reflect', 'replicate' or 'circular'. Default: 'zeros'

- **dilation** (int or tuple, optional) – Spacing between kernel elements. Default: 1

- **groups** (int, optional) – Number of blocked connections from input channels to output channels. Default: 1

- **bias** (bool, optional) – If True, adds a learnable bias to the output. Default: True


https://pytorch.org/docs/stable/generated/torch.nn.Conv2d.html



---

**BATCHNORM2D**

> CLASS: ```torch.nn.BatchNorm2d(num_features, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True, device=None, dtype=None)```

Applies Batch Normalization over a 4D input (a mini-batch of 2D inputs with additional channel dimension) as described in the paper [Batch Normalization: Accelerating Deep Network Training by Reducing Internal Covariate Shift .](https://arxiv.org/abs/1502.03167)

$$y = \frac{x - \mathrm{E}[x]}{ \sqrt{\mathrm{Var}[x] + \epsilon}} * \gamma + \beta$$

The mean and standard-deviation are calculated per-dimension over the mini-batches and **γ** and **β** are learnable parameter vectors of size **C** (where C is the input size). By default, the elements of **γ** are set to 1 and the elements of **β** are set to 0. The standard-deviation is calculated via the biased estimator, equivalent to **torch.var(input, unbiased=False)**.

Also by default, during training this layer keeps running estimates of its computed mean and variance, which are then used for normalization during evaluation. The running estimates are kept with a default momentum of 0.1.

If track_running_stats is set to False, this layer then does not keep running estimates, and batch statistics are instead used during evaluation time as well.

**NOTE**

> This ```momentum``` argument is different from one used in optimizer classes and the conventional notion of momentum. Mathematically, the update rule for running statistics here is $\hat{x}_\text{new} = (1 - \text{momentum}) \times \hat{x} + \text{momentum} \times x_t$, $\hat{x}$ is the estimated statistic and $x_t$ is the new observed value.
​
Because the Batch Normalization is done over the C dimension, computing statistics on (N, H, W) slices, it’s common terminology to call this Spatial Batch Normalization.

**Parameters**

- **num_features** – $C$ from an expected input of size $(N, C, H, W)$

- **eps** – a value added to the denominator for numerical stability. Default: 1e-5

- **momentum** – the value used for the running_mean and running_var computation. Can be set to ```None``` for cumulative moving average (i.e. simple average). Default: 0.1

- **affine** – a boolean value that when set to ```True```, this module has learnable affine parameters. Default: ```True```

- **track_running_stats** – a boolean value that when set to True, this module tracks the running mean and variance, and when set to ```False```, this module does not track such statistics, and initializes statistics buffers ```running_mean``` and ```running_var``` as ```None```. When these buffers are ```None```, this module always uses batch statistics. in both training and eval modes. Default: ```True```

---

**MAXPOOL2D**

> CLASS: ```torch.nn.MaxPool2d(kernel_size, stride=None, padding=0, dilation=1, return_indices=False, ceil_mode=False)```

Applies a 2D max pooling over an input signal composed of several input planes.

In the simplest case, the output value of the layer with input size $(N, C, H, W)$, output $(N, C, H_{out}, W_{out})$ and ```kernel_size``` $(kH, kW)$ can be precisely described as:

$$\begin{aligned} out(N_i, C_j, h, w) ={} & \max_{m=0, \ldots, kH-1} \max_{n=0, \ldots, kW-1} \\ & \text{input}(N_i, C_j, \text{stride[0]} \times h + m, \text{stride[1]} \times w + n) \end{aligned}$$
​
 
If ```padding``` is non-zero, then the input is implicitly zero-padded on both sides for ```padding``` number of points. ```dilation``` controls the spacing between the kernel points. It is harder to describe, but this [link](https://github.com/vdumoulin/conv_arithmetic/blob/master/README.md) has a nice visualization of what ```dilation``` does.

**NOTE**

When ```ceil_mode=True```, sliding windows are allowed to go off-bounds if they start within the left padding or the input. Sliding windows that would start in the right padded region are ignored.

The parameters ```kernel_size```, ```stride```, ```padding```, ```dilation``` can either be:

a single ```int``` – in which case the same value is used for the height and width dimension

a ```tuple``` of two ints – in which case, the first int is used for the height dimension, and the second int for the width dimension

**Parameters**

- **kernel_size** – the size of the window to take a max over

- **stride** – the stride of the window. Default value is kernel_size

- **padding** – implicit zero padding to be added on both sides

- **dilation** – a parameter that controls the stride of elements in the window

- **return_indices** – if True, will return the max indices along with the outputs. Useful for torch.nn.MaxUnpool2d later

- **ceil_mode** – when True, will use ceil instead of floor to compute the output shape

## Define Model

In [15]:
class MNIST_CNN(nn.Module):
    def __init__(self):
        super().__init__()
        # Layer 1: ImageInput Shape=(?, 1, 28, 28)
        # Conv   : ImageOutput Shape=(?, 32, 28, 28)
        # Pool   : ImageOutput Shape=(?, 32, 14, 14)
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size = 3, stride = 1, padding = 1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2)
        )
        # Layer 2: ImageInput Shape=(?, 32, 14, 14)
        # Conv   : ImageOutput Shape=(?, 64, 14, 14)
        # Pool   : ImageOutput Shape=(?, 64, 7, 7)
        self.layer2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size = 3, stride = 1, padding = 1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2),
        )
        # Final Fully Connected
        # 7 x 7 x 64 inputs => 10 outputs
        self.fc = nn.Linear(7 * 7 * 64, 10)
        init.xavier_uniform_(self.fc.weight)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        #print(out.size()) #torch.size([100, 64, 7, 7])
        out = out.view(out.size(0), -1) # Flatten for FC
        out = self.fc(out)
        return out    

In [16]:
model = MNIST_CNN().to(device)

In [17]:
optimizer = optim.Adam(model.parameters(), lr = learning_rate)

## Train

In [18]:
total_batch = len(data_loader)

print('Learning started')

for epoch in range(training_epochs):

    avg_cost = 0

    for X, Y in data_loader:
        # image is already size of (28 x 28), no reshape
        # Label is not one-hot encoded

        X = X.to(device)
        Y = Y.to(device)

        #prediction
        pred = model(X)

        #cost
        cost = F.cross_entropy(pred, Y).to(device)

        #Reduce the cost
        optimizer.zero_grad()
        cost.backward()
        optimizer.step()

        avg_cost += cost

    avg_cost = avg_cost / total_batch

    print('Epoch: {} / {}, cost: {:.6f}'.format(epoch + 1, training_epochs, avg_cost))
print('Learning Finished')

Learning started


C:\Users\buddhalight\envs\buddhalight\lib\site-packages\torch\nn\functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ..\c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


Epoch: 1 / 20, cost: 0.218764
Epoch: 2 / 20, cost: 0.059633
Epoch: 3 / 20, cost: 0.044625
Epoch: 4 / 20, cost: 0.034336
Epoch: 5 / 20, cost: 0.029625
Epoch: 6 / 20, cost: 0.024517
Epoch: 7 / 20, cost: 0.020366
Epoch: 8 / 20, cost: 0.017550
Epoch: 9 / 20, cost: 0.014864
Epoch: 10 / 20, cost: 0.012578
Epoch: 11 / 20, cost: 0.009982
Epoch: 12 / 20, cost: 0.009161
Epoch: 13 / 20, cost: 0.007706
Epoch: 14 / 20, cost: 0.008041
Epoch: 15 / 20, cost: 0.005679
Epoch: 16 / 20, cost: 0.004700
Epoch: 17 / 20, cost: 0.003399
Epoch: 18 / 20, cost: 0.005507
Epoch: 19 / 20, cost: 0.003623
Epoch: 20 / 20, cost: 0.002687
Learning Finished


### Take a Moment!

Why we should ```.view(len(mnist_test), 1, 28 ,28)``` below despite of we didn't do that upon?

=> since ```data_loader``` makes mini batches as size of size([100, 1, 28, 28]), we don't have to do that manually, but we don't use ```data_loader``` below.

In [19]:
#Test model and check accuracy
with torch.no_grad():
    #print(mnist_test.data.shape) => torch.Size([10000, 28, 28])
    X_test = mnist_test.data.view(len(mnist_test), 1, 28, 28).float().to(device)
    Y_test = mnist_test.targets.to(device)

    #prediction
    pred = model(X_test)
    correct_prediction = (torch.argmax(pred, 1) == Y_test)
    accuracy = correct_prediction.float().mean()

    print('Accuracy: ', accuracy.item())

Accuracy:  0.9792999625205994
